In [1]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline


In [43]:
class ParamError(Exception):
    pass

class hp4142b(object):
    
    def __init__(self, inst):
        self.mode = "NORM"
        self.inst = inst
        self.reset()
        self.write("FMT1,1")
        print(self.idn())
                
    def write(self, cmd):
        if not cmd.endswith("\n"):
            cmd = cmd+"\n"
        #print("CMD: %s" % cmd)
        return inst.write(cmd)

    def idn(self):
        self.write("*IDN?")
        try:
            return(inst.read().decode('ascii'))
        except:
            print(gpib_err(inst.ibsta()))
            return None   
    
    def voltage(self, ch, v=0.0, range=0, i_lim=0.001, lim_mode=0):
        return self.write("DV%d,%d,%+.3e,%+.3e,%d\n" % ( ch, range, v, i_lim, lim_mode ))
    
    def pulsed_v(self, ch, base=0.0, pulse=0.0, range=0, i_lim=0.001):
        self.mode="PULSED"
        return self.write("PDV%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, i_lim))
    
    def pulsed_i(self, ch, base=0.0, pulse=0.0, v_lim=1):
        self.mode="PULSED"
        return self.write("PDI%d,%d,%+.3e,%+.3e,%+.3e"%(ch, range, base, pulse, v_lim))
    
    def current(self, ch, i=0.0, range=0, v_lim=1, lim_mode=0):
        return self.write("DI%d,%d,%+.3e,%+.3e,%d\n"%(ch, range, i, v_lim, lim_mode))

    def sweep_i(self, ch, mode='lin', range=0, start=0, stop=1e-3, n=50, v_lim=1, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WI%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, v_lim, p_lim))

    def sweep_v(self, ch, mode='lin', range=0, start=0, stop=1, n=50, i_lim=0.001, p_lim=1):
        self.mode="SWEEP"
        mode_n=1;
        if mode=='log':
            mode_n=2
        if mode=='dlin':
            mode_n=3
        if mode=='dlog':
            mode_n=4
        return self.write("WV%d,%d,%d,%+.3e,%+.3e,%d,%+.3e,%+.3e\n" % (ch, mode_n, range, start, stop, n, i_lim, p_lim))

    def sweep_i_follow(self, ch, range=0, start=0.001, stop=0.001, v_lim=1, p_lim=1):
        return self.write("WSI%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, v_lim, p_lim))
        
    def sweep_v_follow(self, ch, range=0, start=0, stop=1, i_lim=0.001, p_lim=1):
        return self.write("WSV%d,%d,%+.3e,%+.3e,%+.3e,%+.3e"%(ch, range, start, stop, i_lim, p_lim))
    
    def sweep_timing(self, hold=0, delay=0):
        return self.write("WT%.3e,%3e" % (hold, delay))
    
    def search_cfg(self, op="FBpos", meas="searchV", itime=5e-3):
        operations={ 'FBpos': 1,
              'FBneg': 2,
              'ramp_gt': 3,
              'ramp_lt': 4 }
        
        measurements = {'searchV': 1,
                        'searchI': 2,
                        'searchVsenseVI': 3,
                        'searchIsenseVI': 4 }
        
        opcode=operations[op] or None
        mcode = measurements[meas] or None
        
        self.write("ASM%d,%d,%f" % (opcode, mcode, itime))
        
    def search(self, ch, start=0, stop=1, rate=500, i_lim=1e-3):
        G=max(abs(start), abs(stop))
        D=abs(start-stop)
        self.write("ASV%d,%f,%f,%f,%f"%(ch, start, stop, rate, i_lim))
        
    def search_timing(self,hold=0.0, delay=0.0):
        self.write("AT%f,%f"%(hold, delay))
        
    def sense_i(self, ch, v=1, i_target=1e-4, i_lim=1e-3):
        self.write("AVI%d,%f,%f,%f"%(ch, v, i_target, i_lim))
    
    def sense_v(self, ch, i=1e-3, v_target=0.1, v_lim=1):
        self.write("AIV%d,%f,%f,%f"%(ch, i, v_target, v_lim))
    
        
    def connect(self,ch):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CN" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CN%s\n" % ch
        else:
            cmd="CN\n"
        self.write(cmd)

    def zero(self, ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "DZ" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "DZ%s\n" % ch
        else:
            cmd="DZ\n"
        self.write(cmd)
       
    def calibrate(self):
        self.write("CA")
        time.sleep(10)
        self.status()
        print("IBSTA: "+ gpib_err(inst.ibsta()))

        print("Calibrated")
    
    def disconnect(self,ch=None):
        cmd=""
        if ch:
            if type(ch)==type([]):
                cmd = "CL" + (",".join(list(map(str,ch)))) + "\n"
            else:
                cmd = "CL%s\n" % ch
        else:
            cmd="CL\n"
        self.write(cmd)
        
    def opstat(self):
        self.write("LOP?")
        codestring = self.readresult()
        codestring = codestring.replace("\n", "").replace("\r", "")
        codes = codestring[3:].split(",")

        codetotext = { '00': 'Not active',
               '01': 'V source',
               '02': 'I+ source',
               '03': 'I- source',
               '10': 'Both VS limit',
               '11': 'V compliance reached',
               '12': 'I+ compliance reached',
               '13': 'I- compliance reached',
               '20': 'Oscillating',
               '30': 'HVU not settled'
              }
        
        return list(map(lambda c: codetotext[c], codes))
        
        
    def status(self):
        self.write("*STB?")
        st = self.inst.read().decode('ascii')
        print("Status: %s %s" %(st,  statusdecode(int(st))))
        
    def nub(self):
        self.write("NUB?")
        return self.readresult()
    
    def units(self):
        self.write("UNT?")
        return self.readresult()
    
    def reset(self):
        self.write("*RST")
        
    def readresult(self):
        readmore=True
        outdata=""
        ibread=""
        self._resdata = ""
    
        while readmore:
            try:
                ibread=inst.read().decode('ascii')
                if ibread.endswith("\n"):
                    readmore=False
            except:
            
                readmore=False
                
            outdata=outdata+ibread

        print("IBSTA: "+ gpib_err(inst.ibsta()))
        self._resdata = outdata
        return outdata

    def xe(self):
        return self.write("XE")

    
def parseresult(res, channelnames):
    results = {}
    
    errors_reported = []

    items = res.split(",")
    for i in items:
        st=i[0]
        ch=channelnames[int((ord(i[1])-ord('A')+1)/2-1)]
        meas=i[2]
        v=float(i[3:])
        if(st in ('N', 'W', 'E')):
           resname="%s_%s"%(meas, ch)
           resarr=results.get(resname, [])
           resarr.append(v)
           results[resname] = resarr
        else:
            if not(st in errors_reported):
                print("MEAS_ERR: %s" % i)
                errors_reported.append(st)
    return results


hp4142n_status_bits= { 
    'Data ready': 0,
    'Wait': 1,
    'NotUsed': 2,
    'Interlock open': 3,
    'Set ready': 4,
    'Error': 5,
    'RQS': 6,
    'Shut down': 7
    }

gpib_err_bits={
        'DCAS' : 0,
        'DTAS' : 1,
        'LACS' : 2,
        'TACS' : 3,
        'ATN' : 4,
        'CIC' : 5,
        'REM' : 6,
        'LOK' : 7,
        'Complete' : 8,
        'EVENT' : 9,
        'SPOLL' : 10,
        'RQS' : 11,
        'SRQI' : 12,
        'END' : 13,
        'TIMO' : 14,
        'ERR' : 15
};

def gpib_err(bits):
    errs = []
    for err in gpib_err_bits:
        if bits & (2**gpib_err_bits[err]):
            errs.append(err)

    return " ".join(errs)

def statusdecode(bits):
    stat = []
    for err in hp4142n_status_bits:
        if bits & (2**hp4142n_status_bits[err]):
            stat.append(err)

    return " ".join(stat)

In [44]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

print(smu.units())
print(smu.opstat())


HEWLETT PACKARD,4142B,0,4.30

IBSTA: END Complete
0,0;HP41420A,3;0,0;HP41420A,3;0,0;HP41420A,3;HP41425A,0;0,0

IBSTA: END Complete
['Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active', 'Not active']


In [76]:
#hFE measurement

Bch=2
Cch=3
Ic_max=0.2

#2N3904 EBC pinout

E=0
B=2
C=4

channels=['B', 'C']

#Instrument init
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

def meas_npn(Vce=5.0):    
    smu.connect([B,C])
    smu.voltage(C, Vce, i_lim=Ic_max)
    smu.sweep_timing(hold=0.05, delay=0.05)

    smu.sweep_i(B, start=1e-6, stop=0.9e-3, v_lim=4, mode='log')
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    d=parseresult(r, channels)

    return d

def meas_hfe(Ic=1e-3, Vce=1, minhfe=10):
    smu.connect([B,C])
    smu.search(B, start=0, stop=1, rate=200, i_lim=Ic/minhfe) #ASV B, 0, 1, rate=200, i_compl=1e-4
    smu.sense_i(C, v=Vce, i_target=Ic, i_lim=1.2*Ic) #AVI C, Vc=1, Ic_target=1e-3, ic_comp=1.2e-3
    smu.search_timing(hold=10e-3, delay=10e-3)
    smu.search_cfg(op='FBpos', meas="searchIsenseVI") #ASM 1,4,5e-4
    smu.write("MM6")
    smu.xe()
    smu.disconnect()
    r=smu.readresult()
    d=parseresult(r, channels)
    #print(d)
    return d['I_C'][0]/d['I_B'][0]


print(meas_hfe(Ic=1e-3, Vce=1))

HEWLETT PACKARD,4142B,0,4.30

IBSTA: END Complete
111.05438401775804


In [14]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

print(smu.nub())
print(smu.lop())

HEWLETT PACKARD,4142B,0,4.30

IBSTA: END Complete
0

IBSTA: END Complete
LOP00,00,00,00,00,00,00,00



In [72]:
data = []
smu = hp4142b(inst)

for vce in ([1,3,10]):
    d=meas_npn(Vce=float(vce))
    ic=np.array(d['I_C'])
    ib=np.array(d['I_B'])
    vb=np.array(d['V_B'])

    hfe=np.divide(ic, ib)
    trace =  go.Scatter(x=1e3*ic, y=hfe, name='hFE@V_CE=%.2f V'%vce)

    data.append( trace )


HEWLETT PACKARD,4142B,0,4.30

IBSTA: END Complete
IBSTA: END Complete
IBSTA: END Complete


In [73]:
layout = go.Layout(
    title='2N3904 hFE',
    xaxis=dict(
        title='I_C(mA)',
        type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='hFE', 
        range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

In [28]:
E=0
B=2
C=4

channels=['B', 'C']

Ic_max=1e-3
Vce=5.0
Iebo_max=1e-6
Vebo_test=20

#Instrument init
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

def vbceo_npn(Vebo_test=10, Iebo=1e-6, Vce=1, Ic_max=0.001):    
    smu.connect([B,C])
    smu.voltage(C, Vce, Ic_max)
    smu.current(B, -Iebo, v_lim=-Vebo_test)
    smu.write("MM1,2,4\n")
    smu.xe()
    print(smu.opstat())
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    
    d=parseresult(r, channels)

    return d['V_B'][0]
    
print(vbceo_npn(Iebo=1e-6))


HEWLETT PACKARD,4142B,0,4.30

IBSTA: END Complete
['Not active', 'I- source', 'Not active', 'V source', 'Not active', 'Not active', 'Not active', 'Not active']
IBSTA: END Complete
-7.6744


In [6]:
def meas_vce_sat():    
    smu.connect([B,C])
    smu.sweep_i(C, start=1e-6, stop=100e-3, v_lim=30, mode='log')
    smu.sweep_i_follow(B, start=1e-6, stop=10e-3, v_lim=2)
    #smu.current(B,i=1e-3,v_lim=2)
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    
    d=parseresult(r, channels)

    return d



In [7]:
vce_sat=meas_vce_sat()
ic=np.array(vce_sat['I_C'])
vc=np.array(vce_sat['V_C'])
trace =  go.Scatter(x=1e3*ic, y=vc, name='V_CE sat vs I_C')

data=[trace]

layout = go.Layout(
    title='2N3904 VCE sat.',
    xaxis=dict(
        title='I_C(mA)',
        type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='VCE sat.',
        range=[0,0.5],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    

STA: CMPL END


In [8]:
ic=np.array(vce_sat['I_C'])
vc=np.array(vce_sat['V_B'])
trace =  go.Scatter(x=1e3*ic, y=vb, name='V_BE sat vs I_C')

data=[trace]

layout = go.Layout(
    title='2N3904 VCE sat.',
    xaxis=dict(
        title='I_C(mA)',
        type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='VBE sat.',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    

In [57]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)
smu.calibrate()
time.sleep(5)
print("1")
time.sleep(5)
print("2")
time.sleep(5)
print("3")
time.sleep(5)
print("R")
#print(smu.readresult())


HEWLETT PACKARD,4142B,0,4.30

1
2
3
R


In [133]:
channels=['X','Y','A']
A=6

r=None

inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)
#smu.calibrate()

def meas_diode():
    smu.connect([A])
    smu.sweep_i(A, start=1e-8, stop=100e-3, v_lim=3, mode='log', n=50)
    smu.write("MM2,6\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print(r)

    d=parseresult(r, channels)

    return d

def meas_diode_rev(): 
    smu.connect([A])
    smu.sweep_v(A, start=-40, stop=-0.1, i_lim=0.001, n=40)
    smu.write("MM2,6\n")
    smu.sweep_timing(hold=0.05, delay=0.05)
    smu.xe()
#    time.sleep(15)
    smu.zero()
    smu.disconnect()
    smu.status()

    r=smu.readresult()
    #print(r)
    d=parseresult(r, channels)

    return d


diode=meas_diode()
rev=meas_diode_rev()
print("Done")


HEWLETT PACKARD,4142B,0,4.30

IBSTA: Complete END
Status: 0
 
IBSTA: Complete END
Done


In [118]:

tracerev =  go.Scatter(x=np.array(rev['V_A']), y=np.array(rev['I_A']), name='Reverse')
tracefwd = go.Scatter(x=np.array(diode['V_A']), y=np.array(diode['I_A']), name="Forward")

data=[tracerev, tracefwd]
#data= [tracerev]
layout = go.Layout(
    title='Red LED diode forward drop',
    xaxis=dict(
        title='V',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout }) 
    

In [48]:
rev['I_A']

[-1.06814e-08,
 -1.00198e-08,
 -1.7704e-09,
 -8.3544e-09,
 -5.6358e-09,
 -1.9352e-09,
 -8.8592e-09,
 -1.2968e-09,
 -3.3972e-09,
 -6.9222e-09,
 1.3822e-09,
 -5.224e-09,
 -2.8544e-09,
 6.678e-10,
 -2.4192e-09,
 1.7328e-09,
 -5.206e-09,
 2.017e-09,
 -2.0618e-09,
 3.3278e-09]

In [145]:
inst = Gpib.Gpib(0, 2)
smu = hp4142b(inst)

smu.connect([A])
smu.current(A, 1e-3, v_lim=5)
#smu.write("MM1,6\n")
#smu.xe()
time.sleep(4)
smu.current(A, 10e-3, v_lim=5)
time.sleep(2)
smu.disconnect()

HEWLETT PACKARD,4142B,0,4.30

